In [ ]:
import numpy as np
from airo_drake import DualArmScene, add_floor, add_manipulator, add_meshcat, add_wall, finish_build
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.tree import ModelInstanceIndex
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.scenes import add_cloth_competition_dual_ur5e_scene, X_W_LCB_DEFAULT, X_W_RCB_DEFAULT
from airo_drake import X_URBASE_ROSBASE


robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
meshcat.SetCameraPose([-1.5, 0, 1.0], [0, 0, 0])

X_W_LCB = X_W_LCB_DEFAULT
X_W_RCB = X_W_RCB_DEFAULT

(arm_left_index, arm_right_index), (gripper_left_index, gripper_right_index) = add_cloth_competition_dual_ur5e_scene(
    robot_diagram_builder, X_W_LCB, X_W_RCB
)
robot_diagram, context = finish_build(robot_diagram_builder, meshcat)

scene = DualArmScene(robot_diagram, arm_left_index, arm_right_index, gripper_left_index, gripper_right_index, meshcat)

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[
        scene.arm_left_index,
        scene.arm_right_index,
        scene.gripper_left_index,
        scene.gripper_right_index,
    ],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
tangled_joints_left = np.deg2rad([0, -90, -90, -90, 90, 0])
# tangled_joints_right = np.deg2rad([224, -116, -110, -133, 40, 0])
tangled_joints_right = np.deg2rad([144, -90, -125, 30, 120, 10])
tangled_joints = np.concatenate([tangled_joints_left, tangled_joints_right])

plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, tangled_joints)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

In [ ]:
from airo_typing import JointConfigurationType

from cloth_tools.kinematics.inverse_kinematics import post_process_solution


test_start_joints = np.array([0, 0, 0, 0, 0, -3.13])
test_solution = np.array([0, 0, 0, 0, 0, 1.57])
print(post_process_solution(test_solution, test_start_joints))

In [ ]:
from functools import partial

from airo_typing import HomogeneousMatrixType, JointConfigurationType
from cloth_tools.kinematics.constants import TCP_TRANSFORM
from cloth_tools.kinematics.inverse_kinematics import (
    inverse_kinematics_in_world_fn,
    inverse_kinematics_in_world_post_processed_fn,
)
from ur_analytic_ik import ur5e


inverse_kinematics_left_fn = partial(
    inverse_kinematics_in_world_post_processed_fn,
    X_W_CB=X_W_LCB,
    tcp_transform=TCP_TRANSFORM,
    reference_configuration=tangled_joints_left,
)
inverse_kinematics_right_fn = partial(
    inverse_kinematics_in_world_post_processed_fn,
    X_W_CB=X_W_RCB,
    tcp_transform=TCP_TRANSFORM,
    reference_configuration=tangled_joints_right,
)

In [ ]:
from airo_planner import DualArmOmplPlanner

joint_bounds_lower = np.deg2rad([-360, -195, -160, -360, -360, -360])
joint_bounds_upper = np.deg2rad([360, 15, 160, 360, 360, 360])
joint_bounds = joint_bounds_lower, joint_bounds_upper

planner = DualArmOmplPlanner(
    is_state_valid_fn=collision_checker.CheckConfigCollisionFree,
    inverse_kinematics_left_fn=inverse_kinematics_left_fn,
    inverse_kinematics_right_fn=inverse_kinematics_right_fn,
    joint_bounds_left=joint_bounds,
    joint_bounds_right=joint_bounds,
)

In [ ]:
from airo_typing import RotationMatrixType
from airo_drake import visualize_frame
from airo_typing import HomogeneousMatrixType

def hang_in_the_air_tcp_orientation(left: bool) -> RotationMatrixType:
    gripper_forward_direction = np.array([0, -1, 0]) if left else np.array([0, 1, 0])
    Z = gripper_forward_direction / np.linalg.norm(gripper_forward_direction)
    X = np.array([0, 0, 1]) if left else np.array([0, 0, -1])
    Y = np.cross(Z, X)
    return np.column_stack([X, Y, Z])


def hang_in_the_air_tcp_pose(left: bool) -> HomogeneousMatrixType:
    position = np.array([0, 0, 0.9])  # 1 m is too close to a singularity
    gripper_orientation = hang_in_the_air_tcp_orientation(left)

    gripper_pose = np.identity(4)
    gripper_pose[0:3, 0:3] = gripper_orientation
    gripper_pose[0:3, 3] = position
    return gripper_pose

stretch_pose_left = hang_in_the_air_tcp_pose(left=True)
stretch_pose_right = hang_in_the_air_tcp_pose(left=False)

stretch_pose_left[:3, 3] += np.array([-0.4, 0.02, 0])
stretch_pose_right[:3, 3] += np.array([-0.4, -0.02, 0])

visualize_frame(scene.meshcat, "stretch_pose_left", stretch_pose_left, opacity=0.5)
visualize_frame(scene.meshcat, "stretch_pose_right", stretch_pose_right, opacity=0.5)

In [ ]:
path = planner.plan_to_tcp_pose(tangled_joints_left, tangled_joints_right, stretch_pose_left, stretch_pose_right)

In [ ]:
from airo_drake import time_parametrize_toppra, animate_dual_joint_trajectory

trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)
animate_dual_joint_trajectory(scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory)

In [ ]:
path.shape

In [ ]:
from ur_analytic_ik import ur5e

def forward_kinematics_in_world_fn(joints, X_W_CB, tcp_transform):
    X_CB_TCP = ur5e.forward_kinematics_with_tcp(*joints, tcp_transform)
    X_W_TCP = X_W_CB @ X_CB_TCP
    return X_W_TCP


forward_kinematics_left_fn = partial(forward_kinematics_in_world_fn, X_W_CB=X_W_LCB, tcp_transform=TCP_TRANSFORM)
forward_kinematics_right_fn = partial(forward_kinematics_in_world_fn, X_W_CB=X_W_RCB, tcp_transform=TCP_TRANSFORM)

path_left = path[:, :6]
path_right = path[:, 6:]


In [ ]:
tcp_pose_left = forward_kinematics_left_fn(path_left[0])
tcp_pose_right = forward_kinematics_right_fn(path_right[0])

visualize_frame(scene.meshcat, "tcp_pose_left", tcp_pose_left, opacity=0.5)
visualize_frame(scene.meshcat, "tcp_pose_right", tcp_pose_right, opacity=0.5)

print(tangled_joints_right)
print("IK Solutions")
for s in inverse_kinematics_right_fn(tcp_pose_right):
    print(s, "->", post_process_solution(s, tangled_joints_right))

In [ ]:
from airo_typing import InverseKinematicsFunctionType, JointPathType
from pydrake.trajectories import PiecewisePolynomial
from airo_drake import discretize_drake_joint_trajectory

# For dual arm use only
def calculate_tcp_paths(
    path: JointPathType,
    forward_kinematics_left_fn: InverseKinematicsFunctionType,
    forward_kinematics_right_fn: InverseKinematicsFunctionType,
):
    
    # TODO interpolate joint path with drake and then discretize again
    path = np.array(path).squeeze()
    times_dummy = np.linspace(0.0, 1.0, len(path))  # TOPP-RA will calculate the actual times
    joint_trajectory = PiecewisePolynomial.FirstOrderHold(times_dummy, path.T)

    # path_discretized = discretize_drake_joint_trajectory(joint_trajectory)

    joint_positions = []
    times_uniform = np.linspace(joint_trajectory.start_time(), joint_trajectory.end_time(), 100)
    for t in times_uniform:
        position = joint_trajectory.value(t).squeeze()
        joint_positions.append(position)

    path_discretized = np.array(joint_positions)

    tcp_path_left = []
    tcp_path_right = []
    for i in range(len(path_discretized)):
        joints = path_discretized[i]
        tcp_pose_left = forward_kinematics_left_fn(joints[:6])
        tcp_pose_right = forward_kinematics_right_fn(joints[6:])
        tcp_path_left.append(tcp_pose_left)
        tcp_path_right.append(tcp_pose_right)
    return np.array(tcp_path_left), np.array(tcp_path_right)


tcp_path_left, tcp_path_right = calculate_tcp_paths(path, forward_kinematics_left_fn, forward_kinematics_right_fn)

tcp_path_right.shape

In [ ]:
from airo_typing import PosePathType
import numpy as np
from pydrake.geometry import Meshcat


def visualize_pose_path(
    meshcat: Meshcat,
    name: str,
    pose_path: PosePathType,
    length: float = 0.05,
    radius: float = 0.001,
    opacity: float = 1.0,
):
    for i in range(1, len(pose_path)):
        visualize_frame(
            meshcat,
            f"{name}/pose_{i}",
            pose_path[i],
            length=length,
            radius=radius,
            opacity=opacity,
        )


visualize_pose_path(scene.meshcat, "tcp_path_left", tcp_path_left)
visualize_pose_path(scene.meshcat, "tcp_path_right", tcp_path_right)

In [ ]:
scene.meshcat.Delete("tcp_path_left/")
scene.meshcat.Delete("tcp_path_right/")

In [ ]:
from airo_typing import Vector3DArrayType


def extract_positions(pose_path: PosePathType) -> Vector3DArrayType:
    return np.array([pose[:3, 3] for pose in pose_path])

positions_left = extract_positions(tcp_path_left)
positions_right = extract_positions(tcp_path_right)
positions_left.shape, positions_right.shape

In [ ]:
def calculate_cartesian_path_length(positions: Vector3DArrayType) -> float:
    return np.sum(np.linalg.norm(np.diff(positions, axis=0), axis=1))

length_left = calculate_cartesian_path_length(positions_left)
length_right = calculate_cartesian_path_length(positions_right)

length_left, length_right

In [ ]:
len(planner._all_paths)

In [ ]:
for path in planner._all_paths:
    path_left = path[:, :6]
    path_right = path[:, 6:]
    tcp_path_left, tcp_path_right = calculate_tcp_paths(path, forward_kinematics_left_fn, forward_kinematics_right_fn)
    positions_left = extract_positions(tcp_path_left)
    positions_right = extract_positions(tcp_path_right)
    length_left = calculate_cartesian_path_length(positions_left)
    length_right = calculate_cartesian_path_length(positions_right)
    print(length_left, length_right, tcp_path_left.shape)

In [ ]:
def choose_path_with_closest_tcp_positions(paths: list[JointPathType]) -> JointPathType:
    """Chooses the path where the maximum distance between the TCPs is minimized.
    
    This is useful when both arms are holding a piece of cloth that shouldn't be extended too much.
    """
    max_distance = np.inf
    best_path = None
    for path in paths:
        tcp_path_left, tcp_path_right = calculate_tcp_paths(path, forward_kinematics_left_fn, forward_kinematics_right_fn)
        positions_left = extract_positions(tcp_path_left)
        positions_right = extract_positions(tcp_path_right)
        distance = np.max(np.linalg.norm(positions_left - positions_right, axis=1))
        if distance < max_distance:
            max_distance = distance
            best_path = path
    return best_path

best_path = choose_path_with_closest_tcp_positions(planner._all_paths)

print(best_path.shape)

tcp_path_left, tcp_path_right = calculate_tcp_paths(best_path, forward_kinematics_left_fn, forward_kinematics_right_fn)
print(len(tcp_path_left), len(tcp_path_right))
visualize_pose_path(scene.meshcat, "best_tcp_path_left", tcp_path_left)
visualize_pose_path(scene.meshcat, "best_tcp_path_right", tcp_path_right)

positions_left = extract_positions(tcp_path_left)
positions_right = extract_positions(tcp_path_right)
length_left = calculate_cartesian_path_length(positions_left)
length_right = calculate_cartesian_path_length(positions_right)

length_left, length_right